In [2]:
!pip install QuantLib

import QuantLib as ql

def bond_price_and_durations(
    settlement_date_str,
    maturity_date_str,
    coupon_rate,
    yield_rate,
    face_value=100,
    frequency=ql.Semiannual,
    day_count=ql.ActualActual(ql.ActualActual.Bond)
):
    # Convert dates
    settlement_date = ql.DateParser.parseISO(settlement_date_str)
    maturity_date = ql.DateParser.parseISO(maturity_date_str)

    # Calendar
    calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)

    # Schedule
    schedule = ql.Schedule(
        settlement_date,
        maturity_date,
        ql.Period(frequency),
        calendar,
        ql.Following,
        ql.Following,
        ql.DateGeneration.Backward,
        False
    )

    # Bond
    bond = ql.FixedRateBond(
        2,
        face_value,
        schedule,
        [coupon_rate],
        day_count
    )

    # Yield curve
    curve = ql.YieldTermStructureHandle(
        ql.FlatForward(settlement_date, yield_rate, day_count)
    )
    bond.setPricingEngine(ql.DiscountingBondEngine(curve))

    # Prices
    clean = bond.cleanPrice()
    dirty = bond.dirtyPrice()

    # Duration parameters
    compounding = ql.Compounded

    # Macaulay Duration
    macaulay = ql.BondFunctions.duration(
        bond,
        yield_rate,
        day_count,
        compounding,
        frequency,
        ql.Duration.Macaulay,
        settlement_date
    )

    # Modified Duration
    modified = ql.BondFunctions.duration(
        bond,
        yield_rate,
        day_count,
        compounding,
        frequency,
        ql.Duration.Modified,
        settlement_date
    )

    #ou DV01 (bps)     # per 100 notional
    dv01 = ql.BondFunctions.bps(
    bond,
    yield_rate,
    day_count,
    ql.Compounded,
    frequency,
    settlement_date
)

    return {
        "Clean Price": clean,
        "Dirty Price": dirty,
        "Macaulay Duration": macaulay,
        "Modified Duration": modified,
        "DV01 per 100": dv01
    }

In [17]:
bond_price_and_durations(
    settlement_date_str="2025-12-03",
    maturity_date_str="2051-02-08",
    coupon_rate=0.0265,
    yield_rate=0.0535,
)

{'Clean Price': 62.23537461501381,
 'Dirty Price': 62.24993505457424,
 'Macaulay Duration': 16.463138546794003,
 'Modified Duration': 16.03422307941953,
 'DV01 per 100': 0.1376319420371703}

In [4]:
#FIRST MODEL - from YTM to prices

import QuantLib as ql
import pandas as pd

def bond_full_report(settlement_date_str, maturity_date_str, coupon_rate, yield_rate,
                     face_value=100, frequency=ql.Semiannual, day_count=ql.ActualActual(ql.ActualActual.Bond)):
    
    # 1. Dates
    settlement_date = ql.DateParser.parseISO(settlement_date_str)
    maturity_date = ql.DateParser.parseISO(maturity_date_str)
    
    # 2. Calendar & schedule
    calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
    schedule = ql.Schedule(settlement_date, maturity_date, ql.Period(frequency),
                           calendar, ql.Following, ql.Following, ql.DateGeneration.Backward, False)
    
    # 3. Bond object
    bond = ql.FixedRateBond(2, face_value, schedule, [coupon_rate], day_count)
    
    # 4. Yield curve
    curve = ql.YieldTermStructureHandle(ql.FlatForward(settlement_date, yield_rate, day_count))
    bond.setPricingEngine(ql.DiscountingBondEngine(curve))
    
    # 5. Prices
    clean = bond.cleanPrice()
    dirty = bond.dirtyPrice()
    accrued = bond.accruedAmount()
    
    # 6. Risk metrics
    comp = ql.Compounded
    macaulay = ql.BondFunctions.duration(bond, yield_rate, day_count, comp, frequency, ql.Duration.Macaulay, settlement_date)
    modified = ql.BondFunctions.duration(bond, yield_rate, day_count, comp, frequency, ql.Duration.Modified, settlement_date)
    dv01 = ql.BondFunctions.bps(bond, yield_rate, day_count, comp, frequency, settlement_date)
    convexity = ql.BondFunctions.convexity(bond, yield_rate, day_count, comp, frequency, settlement_date)
    
    # 7. Cash flow schedule
    cf_dates = [cf.date() for cf in bond.cashflows() if cf.date() > settlement_date]
    cf_amounts = [cf.amount() for cf in bond.cashflows() if cf.date() > settlement_date]
    cashflows = pd.DataFrame({"Date": cf_dates, "Amount": cf_amounts})
    
    # 8. Compile report
    report = {
        "Clean Price": clean,
        "Dirty Price": dirty,
        "Accrued Interest": accrued,
        "Macaulay Duration": macaulay,
        "Modified Duration": modified,
        "DV01 per 100": dv01,
        "Convexity": convexity,
        "Cash Flows": cashflows
    }
    
    return report

In [16]:
report = bond_full_report(
    settlement_date_str="2025-12-05", # T+2 settlement
    maturity_date_str="2051-02-08",
    coupon_rate=0.0265,
    yield_rate=0.0535, #! is Yield to maturity
    # Add here frequency=ql.Annual to change from semiannual to annual payments
)

print("Clean Price:", report["Clean Price"])
print("Dirty Price:", report["Dirty Price"])
print("Macaulay Duration:", report["Macaulay Duration"])
print("Modified Duration:", report["Modified Duration"])
print("DV01 per 100:", report["DV01 per 100"])
print(f"Convexity: {report['Convexity']:.4f}")
print("\nCash Flows:\n", report["Cash Flows"])

Clean Price: 62.21725793192529
Dirty Price: 62.21725793192529
Macaulay Duration: 16.461376419602313
Modified Duration: 16.032506861068725
DV01 per 100: 0.13757747834975662
Convexity: 343.8908

Cash Flows:
                    Date      Amount
0    February 9th, 2026    0.480495
1     August 10th, 2026    1.325000
2    February 8th, 2027    1.325000
3      August 9th, 2027    1.325000
4    February 8th, 2028    1.325000
5      August 8th, 2028    1.325000
6    February 8th, 2029    1.325000
7      August 8th, 2029    1.325000
8    February 8th, 2030    1.325000
9      August 8th, 2030    1.325000
10  February 10th, 2031    1.325000
11     August 8th, 2031    1.325000
12   February 9th, 2032    1.325000
13     August 9th, 2032    1.325000
14   February 8th, 2033    1.325000
15     August 8th, 2033    1.325000
16   February 8th, 2034    1.325000
17     August 8th, 2034    1.325000
18   February 8th, 2035    1.325000
19     August 8th, 2035    1.325000
20   February 8th, 2036    1.325000
21

In [6]:
#SECOND MODEL - backwardation knowing the market price to get to YTM

import QuantLib as ql
import pandas as pd

def bond_ytm_from_price(settlement_date_str, maturity_date_str, coupon_rate, market_price,
                        is_clean=True, face_value=100, frequency=ql.Semiannual, 
                        day_count=ql.ActualActual(ql.ActualActual.Bond), today_date_str=None,
                        issue_date_str=None):
    """
    Calculate yield to maturity from market price and return full bond analytics
    
    Parameters:
    -----------
    settlement_date_str : str
        Settlement date in ISO format (YYYY-MM-DD)
    maturity_date_str : str
        Maturity date in ISO format (YYYY-MM-DD)
    coupon_rate : float
        Annual coupon rate (e.g., 0.05 for 5%)
    market_price : float
        Observed market price (clean or dirty)
    is_clean : bool
        True if market_price is clean, False if dirty (default True)
    face_value : float
        Face value of bond (default 100)
    frequency : ql.Frequency
        Payment frequency (default Semiannual)
    day_count : ql.DayCounter
        Day count convention (default ActualActual)
    today_date_str : str
        Today's date for evaluation (default: uses settlement_date)
    issue_date_str : str
        Bond issue date in ISO format (default: None, will use first coupon date before settlement)
        
    Returns:
    --------
    dict : Yield to maturity and comprehensive bond analytics
    """
    
    # 1. Dates
    settlement_date = ql.DateParser.parseISO(settlement_date_str)
    maturity_date = ql.DateParser.parseISO(maturity_date_str)
    
    # Determine issue date
    if issue_date_str is not None:
        issue_date = ql.DateParser.parseISO(issue_date_str)
    else:
        # If no issue date provided, calculate backwards from maturity
        # Assume bond was issued on a coupon date
        if frequency == ql.Semiannual:
            issue_date = maturity_date - ql.Period(5, ql.Years)
        elif frequency == ql.Annual:
            issue_date = maturity_date - ql.Period(5, ql.Years)
        else:
            issue_date = settlement_date
    
    # Set today's date (if not provided, use settlement date)
    if today_date_str is None:
        today_date = settlement_date
    else:
        today_date = ql.DateParser.parseISO(today_date_str)
    
    # CRITICAL: Set QuantLib's global evaluation date
    ql.Settings.instance().evaluationDate = today_date
    
    # 2. Calendar & schedule - USE ISSUE DATE AS START
    calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
    schedule = ql.Schedule(issue_date, maturity_date, ql.Period(frequency),
                           calendar, ql.Following, ql.Following, ql.DateGeneration.Backward, False)
    
    # 3. Bond object
    bond = ql.FixedRateBond(2, face_value, schedule, [coupon_rate], day_count)
    
    # 4. Calculate accrued interest
    accrued = bond.accruedAmount(settlement_date)
    
    # 5. Convert to clean price if dirty price given
    if is_clean:
        clean_price = market_price
    else:
        clean_price = market_price - accrued
    
    # 6. Calculate yield from clean price
    comp = ql.Compounded
    # Convert price to BondPrice object (Clean price type)
    bond_price = ql.BondPrice(clean_price, ql.BondPrice.Clean)
    yield_rate = ql.BondFunctions.bondYield(bond, bond_price, day_count, comp, frequency, settlement_date)
    
    # 7. Set up yield curve with calculated yield
    curve = ql.YieldTermStructureHandle(ql.FlatForward(settlement_date, yield_rate, day_count))
    bond.setPricingEngine(ql.DiscountingBondEngine(curve))
    
    # 8. Prices (for verification)
    clean = bond.cleanPrice()
    dirty = bond.dirtyPrice()
    
    # 9. Risk metrics
    macaulay = ql.BondFunctions.duration(bond, yield_rate, day_count, comp, frequency, ql.Duration.Macaulay, settlement_date)
    modified = ql.BondFunctions.duration(bond, yield_rate, day_count, comp, frequency, ql.Duration.Modified, settlement_date)
    dv01 = ql.BondFunctions.bps(bond, yield_rate, day_count, comp, frequency, settlement_date)
    convexity = ql.BondFunctions.convexity(bond, yield_rate, day_count, comp, frequency, settlement_date)
    
    # 10. Cash flow schedule
    cf_dates = [cf.date() for cf in bond.cashflows() if cf.date() > settlement_date]
    cf_amounts = [cf.amount() for cf in bond.cashflows() if cf.date() > settlement_date]
    cashflows = pd.DataFrame({"Date": cf_dates, "Amount": cf_amounts})
    
    # 11. Compile report
    report = {
        "Issue Date": str(issue_date),
        "Today's Date": str(today_date),
        "Settlement Date": str(settlement_date),
        "Maturity Date": str(maturity_date),
        "Yield to Maturity": yield_rate,
        "Input Market Price": market_price,
        "Input Price Type": "Clean" if is_clean else "Dirty",
        "Calculated Clean Price": clean,
        "Calculated Dirty Price": dirty,
        "Accrued Interest": accrued,
        "Price Verification (Clean)": abs(clean_price - clean),
        "Macaulay Duration": macaulay,
        "Modified Duration": modified,
        "DV01 per 100": dv01,
        "Convexity": convexity,
        "Cash Flows": cashflows
    }
    
    return report

In [18]:
report = bond_ytm_from_price(
    today_date_str="2025-12-03",      # Today
    settlement_date_str="2025-12-05", # T+2 settlement
    maturity_date_str="2060-08-20",
    coupon_rate=0.0255,
    market_price=56.78,
    is_clean=True,
    issue_date_str="2021-02-08"  # ← Bond issued 5 years before maturity
)

# Display the results
print(f"Issue Date: {report['Issue Date']}")
print(f"Settlement: {report['Settlement Date']}")
print(f"Maturity: {report['Maturity Date']}")
print(f"\nInput Market Price: ${report['Input Market Price']:.4f} ({report['Input Price Type']})")
print(f"Calculated Clean Price: ${report['Calculated Clean Price']:.4f}")
print(f"Calculated Dirty Price: ${report['Calculated Dirty Price']:.4f}")
print(f"Accrued Interest: ${report['Accrued Interest']:.4f}")
print(f"Difference (Dirty - Clean): ${report['Calculated Dirty Price'] - report['Calculated Clean Price']:.4f}")
print(f"\nYield to Maturity: {report['Yield to Maturity']*100:.4f}%")
print(f"Modified Duration: {report['Modified Duration']:.4f}")
print(f"Macaulay Duration: {report['Macaulay Duration']:.4f}")
print(f"DV01 per 100: ${report['DV01 per 100']:.6f}")
print(f"Convexity: {report['Convexity']:.4f}")


Issue Date: February 8th, 2021
Settlement: December 5th, 2025
Maturity: August 20th, 2060

Input Market Price: $56.7800 (Clean)
Calculated Clean Price: $55.9364
Calculated Dirty Price: $56.6778
Accrued Interest: $0.7414
Difference (Dirty - Clean): $0.7414

Yield to Maturity: 5.2783%
Modified Duration: 18.2861
Macaulay Duration: 18.7687
DV01 per 100: $0.160987
Convexity: 498.5399
